### Bloco 1: Importação de bibliotecas
Este bloco importa as bibliotecas necessárias para a manipulação de vídeo e áudio:

- **`moviepy.editor`**: Para edição de vídeos e áudio.
- **`numpy`**: Para manipulação de arrays e processamento numérico.
- **`matplotlib.pyplot`**: Embora não seja utilizada neste exemplo, pode ser usada para visualização de dados se necessário.

In [1]:

import moviepy.editor as mpe
import numpy as np
import matplotlib.pyplot as plt


### Bloco 2: Carregamento do vídeo
Este bloco carrega o arquivo de vídeo que será editado. 

- **`video_path`**: Substitua pelo caminho do arquivo de vídeo que você deseja processar.
- **`mpe.VideoFileClip`**: Carrega o vídeo para que ele possa ser manipulado.

In [ ]:

video_path = "video_inicial.mp4"
video = mpe.VideoFileClip(video_path)


### Bloco 3: Inversão de imagem no eixo a cada 20 segundos
Este bloco aplica uma inversão da imagem no eixo vertical a cada 20 segundos no vídeo:

- **`invert_axis`**: Define uma função que inverte a imagem verticalmente.
- **Loop**: Divide o vídeo em segmentos de 20 segundos e aplica a inversão em segmentos ímpares.
- **`mpe.concatenate_videoclips`**: Junta todos os segmentos em um único clipe.

In [ ]:
def invert_axis(clip):
    return clip.fl_image(lambda frame: np.flipud(frame)) 

chunks = []
for i in range(0, int(video.duration), 20):
    chunk = video.subclip(i, min(i + 20, video.duration))
    if (i // 20) % 2 == 1:  
        chunk = invert_axis(chunk)
    chunks.append(chunk)

video_inverted = mpe.concatenate_videoclips(chunks)


### Bloco 4: Gradativa redução de volume
Este bloco reduz gradativamente o volume do áudio do vídeo:

- **`gradual_volume`**: Aplica uma função para reduzir o volume do áudio gradativamente.
- O volume atinge zero nos últimos 10 segundos.
- **Nota**: Caso o vídeo não tenha áudio, o código continuará sem aplicar modificações.

In [ ]:
def gradual_volume(clip):
    def audio_fadeout(get_audio, t):
        audio = get_audio(t)  
        volume_factor = np.clip(1 - t / (clip.duration - 10), 0, 1)  
        if audio.ndim > 1:  
            volume_factor = volume_factor[:, np.newaxis]  
        return audio * volume_factor

    
    if clip.audio is not None:
        new_audio = clip.audio.fl(audio_fadeout)
        return clip.set_audio(new_audio)
    else:
        print("O clipe não possui áudio.")
        return clip

video_with_audio = gradual_volume(video_inverted)


### Bloco 5: Corte e reposicionamento de trecho
Este bloco realiza as seguintes operações:

1. Corta um trecho de 20 segundos (entre 60 e 80 segundos) do vídeo.
2. Remove esse trecho do vídeo original.
3. Adiciona o trecho cortado ao final do vídeo.

- **`subclip`**: Cria subclipes do vídeo com os trechos desejados.
- **`mpe.concatenate_videoclips`**: Junta os trechos editados.

In [ ]:
part_to_cut = video_with_audio.subclip(60, 80)
remaining_video = mpe.concatenate_videoclips([
    video_with_audio.subclip(0, 60),
    video_with_audio.subclip(80, video_with_audio.duration)
])

final_video = mpe.concatenate_videoclips([remaining_video, part_to_cut])


### Bloco 6: Exportação do vídeo final
Este bloco salva o vídeo final no formato MP4 com as edições realizadas:

- **`write_videofile`**: Exporta o vídeo editado.
- **Parâmetros**:
  - **`codec="libx264"`**: Codec para compressão de vídeo.
  - **`audio_codec="aac"`**: Codec para compressão de áudio.
  - **`temp_audiofile`**: Arquivo temporário para processar o áudio.
  - **`remove_temp`**: Remove o arquivo temporário após a exportação.

In [ ]:
output_path = "video_final.mp4"

final_video.write_videofile(
    output_path,
    codec="libx264",
    audio_codec="aac",
    temp_audiofile="temp_audio.m4a",  
    remove_temp=True                 
)

Moviepy - Building video video_final.mp4.
MoviePy - Writing audio in temp_audio.m4a


MoviePy - Done.
Moviepy - Writing video video_final.mp4



Moviepy - Done !
Moviepy - video ready video_final.mp4
